## LSTM Network Training

In [14]:
import os
import warnings
import pandas as pd

from src.StockPredictionConfig import StockPredictionConfig
from src.StockDataProcessor import StockDataProcessor
from src.LongShortTermMemory import LSTMModel

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or '3' to suppress all messages

# Suppress other warnings
warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")

from datetime import timedelta

In [ ]:
# FOLDER_PREFIX = "data/comb/"
# STOCK_START_DATE = pd.to_datetime("2017-06-07 15:59:00")
# STOCK_VALIDATION_DATE = pd.to_datetime("2024-06-09 09:30:00")
# STOCK_END_DATE = pd.to_datetime("2024-06-12 15:59:00")
# TIME_STEPS = 60
# PREDICTION_RANGE = 60 # *24

In [15]:
# FOLDER_PREFIX = "data/min/"
# STOCK_START_DATE = pd.to_datetime("2024-05-24 09:30:00")
# STOCK_VALIDATION_DATE = pd.to_datetime("2024-06-09 09:30:00")
# STOCK_END_DATE = pd.to_datetime("2024-06-12 15:59:00")
# TIME_STEPS = 60
# PREDICTION_RANGE = 60

In [16]:
FOLDER_PREFIX = "data/day/"
STOCK_START_DATE = pd.to_datetime("2017-06-07")
STOCK_VALIDATION_DATE = pd.to_datetime("2022-06-07")
STOCK_END_DATE = pd.to_datetime("2024-06-07")
TIME_STEPS = 1
PREDICTION_RANGE = 5

In [17]:
# import secrets
# TODAY_RUN = datetime.today().strftime(DATE_FORMAT)
# TOKEN = STOCK_TICKER + '_' + TODAY_RUN + '_' + secrets.token_hex(16)

In [18]:
DATE_FORMAT = "%Y-%m-%d"
STOCK_TICKER = "GOOG"
EPOCHS = 100
BATCH_SIZE = 10
TOKEN = "GOOG"
RUN_FOLDER = f"{FOLDER_PREFIX}{TOKEN}/"
WORK_DIR = os.path.join(os.getcwd(), RUN_FOLDER)
CSV_FILE = f"{WORK_DIR}data.csv"

print('Ticker: ' + STOCK_TICKER)
print('Start Date: ' + STOCK_START_DATE.strftime(DATE_FORMAT))
print('Validation Date: ' + STOCK_VALIDATION_DATE.strftime(DATE_FORMAT))
print('Test Run Folder: ' + RUN_FOLDER)

PROJECT_FOLDER = os.path.join(os.getcwd(), RUN_FOLDER)
if not os.path.exists(PROJECT_FOLDER):
    os.makedirs(PROJECT_FOLDER)

Ticker: GOOG
Start Date: 2017-06-07
Validation Date: 2022-06-07
Test Run Folder: data/day/GOOG/


In [19]:
stock_config = StockPredictionConfig(STOCK_TICKER, 
                                   STOCK_START_DATE,
                                   STOCK_END_DATE,
                                   STOCK_VALIDATION_DATE, 
                                   PROJECT_FOLDER, 
                                   EPOCHS,
                                   TIME_STEPS,
                                   TOKEN,
                                   BATCH_SIZE)


In [20]:
(x_train, y_train), (x_test, y_test), (training_data, test_data) = StockDataProcessor.load_csv_transform_to_numpy(TIME_STEPS, CSV_FILE, STOCK_VALIDATION_DATE)

mean: [0.32731816]
max: 0.9999999999999999
min: 0.0
Std dev: [0.30543526]


In [ ]:
# Execute Deep Learning model
lstm = LSTMModel(stock_config.project_folder)
lstm.Train(stock_config, x_train, y_train, x_test, y_test, training_data, test_data)

## Prediction

In [21]:
data = pd.read_csv(CSV_FILE)

latest_close_price = data['Close'].iloc[-1]
latest_date = data['Datetime'].iloc[-1]

print(f"{latest_close_price}: latest_close_price")
print(f"{latest_date}: latest_date")

start_date = pd.to_datetime(latest_date) + timedelta(1)
end_date = pd.to_datetime(start_date) + timedelta(PREDICTION_RANGE)

print(f"{start_date}: start_date")
print(f"{end_date}: end_date")

175.9499969482422: latest_close_price
2024-06-07: latest_date
2024-06-08 00:00:00: start_date
2024-06-13 00:00:00: end_date


In [ ]:
lstm.Infer(start_date, end_date, latest_close_price, WORK_DIR, TIME_STEPS, STOCK_TICKER, stock_config.CURRENCY)

In [12]:
from datetime import datetime
def calc_date_range(start_date: datetime, 
                    end_date: datetime) -> list[datetime]:
    """
    Calculates a list of dates between the start and end dates.

    Args:
        start_date: The start date.
        end_date: The end date.

    Returns:
        list[datetime]: A list of dates between the start and end dates.
    """

    is_start_day_is_full_day = True if (start_date.hour == start_date.minute == 0) else False
    is_end_day_is_full_day = True if (end_date.hour == end_date.minute == 0) else False
    is_day_interval = is_start_day_is_full_day and is_end_day_is_full_day
    date_range = []
    if (not is_day_interval):
        my_range = (end_date - start_date).total_seconds() / 60
        for n in range(int(my_range)):
            date_range.append(start_date + timedelta(minutes=n))
    else:
        my_range = (end_date - start_date).days
        for n in range(int(my_range)):
            date_range.append(start_date + timedelta(days=n))
    return date_range

In [13]:
calc_date_range(start_date, end_date)

[Timestamp('2024-06-08 00:00:00'),
 Timestamp('2024-06-09 00:00:00'),
 Timestamp('2024-06-10 00:00:00'),
 Timestamp('2024-06-11 00:00:00'),
 Timestamp('2024-06-12 00:00:00')]